### Slicer Licenses

Copy `license-<proj>.txt` files in `slicer-licenses` directory then run the notebook.

See https://drive.google.com/drive/folders/1sALgJk0ROnTkkYIFm050c_8GHc3x2M-7

In [1]:
!pip install what_the_license ipy_table

In [2]:
import glob
import os
import re
import what_the_license

from pprint import pprint as pp

projects_per_license_type = {}

licenses_per_project = {}

license_mapping = {'MIT License': 'The MIT License'}

for license_file in sorted(glob.glob("./slicer-licenses/*")):
    project_name = os.path.basename(os.path.splitext(license_file)[0])[len("license-"):]
    license_name = None
    
    with open(license_file) as content:
        if "3D Slicer Contribution and Software License Agreement" in "".join(content.readlines()):
            license_name = "3D Slicer"
            licenses = [{"license": license_name}]

    if not license_name:
        with open(license_file) as content:
            licenses = what_the_license.wtl(content)
            if licenses:
                license_name = licenses[0]["license"]

    if not license_name:
        with open(license_file) as content:
            found_license = re.match(r"^.+ is distributed under the terms of (.+)$", content.readline())
            if found_license:
                license_name = found_license.group(1)
                license_name = license_mapping.get(license_name, license_name)
                licenses = [{"license": license_name}]
    
    if not license_name:
        print("Failed to extract %s license" % project_name)
        continue
    
    if license_name not in projects_per_license_type:
        projects_per_license_type[license_name] = [project_name]
    else:
        projects_per_license_type[license_name].append(project_name)
            
    licenses_per_project[project_name] = licenses

            
#pp(projects_per_license_type)

project_and_license = []
for license, projects in projects_per_license_type.items():
    for project in projects:
        project_and_license.append((project, license))

project_and_license = sorted(project_and_license, key=lambda item: item[0])

project_and_license.insert(0, ("Project Name", "License"))
        
from ipy_table import apply_theme, make_table
make_table(project_and_license)
apply_theme('basic')

Project Name,License
3DSlicer,3D Slicer
BRAINSTools,"Apache License Version 2.0, January 2004"
CTK,"Apache License Version 2.0, January 2004"
CTKAPPLAUNCHER,"Apache License Version 2.0, January 2004"
CTKAppLauncherLib,"Apache License Version 2.0, January 2004"
CompareVolumes,3D Slicer
DCMTK,BSD 3-Clause License
DataStore,3D Slicer
ITK,"Apache License Version 2.0, January 2004"
LandmarkRegistration,3D Slicer
